# Compute S

In [1]:
import numpy as np
import pandas as pd
import xarray as xr

import dask.dataframe as dd
import os
from glob import glob

import matplotlib.pyplot as plt

import histlib.matchup as match
from  histlib.diagnosis import build_matchup_dataframe, synthetic_figure, path_csv, put_fig_letter, build_diagnostic
from histlib.cstes import labels, zarr_dir, matchup_dir, c0

/home1/datahome/mdemol/.miniconda3/envs/histenv2/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.scheduler.transition-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(
/home1/datahome/mdemol/.miniconda3/envs/histenv2/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.comm.recent-messages-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(


In [12]:
if True:
    from dask.distributed import Client
    from dask_jobqueue import PBSCluster
    #cluster = PBSCluster(cores=56, processes=28, walltime='04:00:00')
    #cluster = PBSCluster(cores=7, processes=7, walltime='04:00:00')
    cluster = PBSCluster(cores=20, processes=20, walltime='04:00:00')
    w = cluster.scale(jobs=1)
else:
    from dask.distributed import Client, LocalCluster
    cluster = LocalCluster()

client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.148.1.29:8787/status,
Dashboard: http://10.148.1.29:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.1.29:54837,Workers: 0
Dashboard: http://10.148.1.29:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [66]:
cluster.close()

________
# Mean

In [19]:
def id_from_comb(comb):
    return '__'.join([comb[key] for key in comb])

def onel_onecomb_mean_dataset(l, comb, drifter_type, drogue_status, DL, DT):
    paths = [f'/home/datawork-lops-osi/aponte/margot/historical_coloc_ok/matchup/matchup_{l}.zarr'] + glob(os.path.join(f'/home/datawork-lops-osi/aponte/margot/historical_coloc_ok/cutoff_matchup/cutoff_matchup_{l}*.zarr'))
    
    ds = xr.merge([xr.open_dataset(path) for path in paths])[[comb[key] for key in comb]+['drogue_status']]
    ds['drifter_type'] = l.split('_')[0]
    ds['drogue_status'] = ds['drogue_status'].astype('bool')
    ds = ds.reset_coords(['lon', 'lat'])
    
    dfm = ds.to_dataframe()
    
    if drogue_status != 'both':
        dfm = dfm.where(dfm.drogue_status==drogue_status).dropna()
    if DL :
        dfm =dfm.where(dfm.alti___distance<DL).dropna()
    if DT : 
        dfm =dfm.where(dfm.alti___time_difference<DT).dropna()
        
    dfm = build_diagnostic(dfm, comb).drop(columns = ['drogue_status','alti___distance', 'alti___time_difference', 'drifter_type','time']).set_index('obs')

    dim = len(dfm)
    ds = dfm.to_xarray().mean('obs')

    ds['nb_coloc'] = dim
    ds['drifter_sat_year']=l
    ds = ds.expand_dims('drifter_sat_year')
    ds = ds.set_coords('drifter_sat_year')
    ds['id_comb'] = id_from_comb(comb)
    ds = ds.expand_dims('id_comb')
    ds = ds.set_coords('id_comb')
    
    ## ATTRS
    for v in ds.keys() :
        #ds[v].attrs =dsm[v].attrs
        ds[v].attrs['units']=r'$m^2.s^{-4}$'
    return ds

def mean_dataset(combinations, drifter_type = 'both', drogue_status=True, DL=None, DT=None):
    from histlib.cstes import labels
    DS =[]
    for l in labels :
        DS0 = []
        for comb in combinations :
            ds = onel_onecomb_mean_dataset(l, comb, drifter_type, drogue_status, DL, DT)
            DS0.append(ds)
        DS.append(xr.concat(DS0, dim = 'id_comb'))
        print(l)
    return xr.concat(DS, dim='drifter_sat_year')
        
        
    


In [22]:
comb_alg = {'acc':'drifter_acc_x_25','cor':'drifter_coriolis_x_25','ggx':'alti_ggx_adt_filtered_ocean_tide_internal_tide','wd':'es_cstrio_z15_drifter_wd_x'}
comb_05 = {'acc':'drifter_acc_x_05','cor':'drifter_coriolis_x_05','ggx':'alti_ggx_adt_filtered_ocean_tide_internal_tide','wd':'es_cstrio_z15_drifter_wd_x'}
dsm = onel_onecomb_mean_dataset(labels[0], comb_alg, drifter_type = 'both', drogue_status=True, DL=25e3, DT=30*60)
ds = mean_dataset([comb_alg,comb_05], drifter_type = 'both', drogue_status=True, DL=25e3, DT=30*60)

gps_Jason-3_2020
argos_Jason-3_2020
gps_SARAL_2020
argos_SARAL_2020
gps_Cryosat-2_2020
argos_Cryosat-2_2020



KeyboardInterrupt



________
# Lonlat

In [26]:
def onel_onecomb_lonlat_dataset(l, comb, drifter_type, drogue_status, DL, DT, dl):
    paths = [f'/home/datawork-lops-osi/aponte/margot/historical_coloc_ok/matchup/matchup_{l}.zarr'] + glob(os.path.join(f'/home/datawork-lops-osi/aponte/margot/historical_coloc_ok/cutoff_matchup/cutoff_matchup_{l}*.zarr'))
    ds = xr.merge([xr.open_dataset(path) for path in paths])[[comb[key] for key in comb]+['drogue_status']]
    ds['drifter_type'] = l.split('_')[0]
    ds['drogue_status'] = ds['drogue_status'].astype('bool')
    ds = ds.reset_coords(['lon', 'lat'])
    
    dfm = ds.to_dask_dataframe().set_index('obs')
    
    if drogue_status != 'both':
        dfm = dfm.where(dfm.drogue_status==drogue_status).dropna()
    if DL :
        dfm =dfm.where(dfm.alti___distance<DL).dropna()
    if DT : 
        dfm =dfm.where(dfm.alti___time_difference<DT).dropna()
        
    dfm = build_diagnostic(dfm, comb).drop(columns = ['drogue_status','alti___distance', 'alti___time_difference', 'drifter_type','time'])

    dfm["latbin"] = (dfm.lat // dl) * dl
    dfm["lonbin"] = (dfm.lon // dl) * dl
    dd = dfm.groupby(["latbin", "lonbin"]).mean()
    #count
    dnb = dfm.reset_index()[['obs', 'latbin', 'lonbin']].groupby(["latbin", "lonbin"]).count().obs.compute().to_xarray()
    dsms = dd.compute().to_xarray()

    #attrs
    for v in list(dsms.variables) :
        if v in ds :
            dsms[v].attrs = ds[v].attrs
    #merge
    dso = xr.merge([dsms, dnb.rename('nb_coloc_bin')])
    dso['drifter_sat_year']=l
    dso = dso.expand_dims('drifter_sat_year')
    dso = dso.set_coords('drifter_sat_year')
    # center lon, lat bins + reindex to have same for all
    lon_bins, lat_bins = np.arange(-180, 180, dl), np.arange(-90, 90, dl)
    dso = dso.reindex({'lonbin':lon_bins, 'latbin':lat_bins})
    dso['lonbin'] = dso['lonbin']+dl/2
    dso['latbin'] = dso['latbin']+dl/2

    dso['id_comb'] = id_from_comb(comb)
    dso = dso.expand_dims('id_comb')
    dso = dso.set_coords('id_comb')
    
    ## ATTRS
    #for v in ds.keys() :
    #    ds[v].attrs =dsm[v].attrs
    return dso

def lonlat_dataset(combinations, drifter_type = 'both', drogue_status=True, DL=None, DT=None, dl=5):
    from histlib.cstes import labels
    DS =[]
    for l in labels :
        DS0 = []
        for comb in combinations :
            ds = onel_onecomb_lonlat_dataset(l, comb, drifter_type, drogue_status, DL, DT, dl)
            DS0.append(ds)
        DS.append(xr.concat(DS0, dim = 'id_comb'))
        print(l)
    return xr.concat(DS, dim='drifter_sat_year')
    


In [27]:
comb_alg = {'acc':'drifter_acc_x_25','cor':'drifter_coriolis_x_25','ggx':'alti_ggx_adt_filtered_ocean_tide_internal_tide','wd':'es_cstrio_z15_drifter_wd_x'}
comb_05 = {'acc':'drifter_acc_x_05','cor':'drifter_coriolis_x_05','ggx':'alti_ggx_adt_filtered_ocean_tide_internal_tide','wd':'es_cstrio_z15_drifter_wd_x'}
dsm = onel_onecomb_lonlat_dataset(labels[0], comb_alg, drifter_type = 'both', drogue_status=True, DL=25e3, DT=30*60, dl=5)
ds = mean_dataset([comb_alg,comb_05], drifter_type = 'both', drogue_status=True, DL=25e3, DT=30*60)

In [28]:
dsm

<xarray.Dataset>
Dimensions:           (lonbin: 72, latbin: 36, drifter_sat_year: 1, id_comb: 1)
Coordinates:
  * lonbin            (lonbin) float64 -177.5 -172.5 -167.5 ... 172.5 177.5
  * latbin            (latbin) float64 -87.5 -82.5 -77.5 ... 77.5 82.5 87.5
  * drifter_sat_year  (drifter_sat_year) <U16 'gps_Jason-3_2020'
  * id_comb           (id_comb) <U115 'drifter_acc_x_25__drifter_coriolis_x_2...
Data variables: (12/35)
    obs               (id_comb, drifter_sat_year, latbin, lonbin) float64 nan...
    acc               (id_comb, drifter_sat_year, latbin, lonbin) float64 nan...
    cor               (id_comb, drifter_sat_year, latbin, lonbin) float64 nan...
    ggx               (id_comb, drifter_sat_year, latbin, lonbin) float64 nan...
    wd                (id_comb, drifter_sat_year, latbin, lonbin) float64 nan...
    lat               (id_comb, drifter_sat_year, latbin, lonbin) float64 nan...
    ...                ...
    COR               (id_comb, drifter_sat_year, latbin, lonbin) float64 nan...
    GGX               (id_comb, drifter_sat_year, latbin, lonbin) float64 nan...
    WD                (id_comb, drifter_sat_year, latbin, lonbin) float64 nan...
    S                 (id_comb, drifter_sat_year, latbin, lonbin) float64 nan...
    sigma             (id_comb, drifter_sat_year, latbin, lonbin) float64 nan...
    nb_coloc_bin      (id_comb, drifter_sat_year, latbin, lonbin) float64 nan...

________
# Dist

In [37]:
def onel_onecomb_dist_dataset(l, comb, drifter_type, drogue_status, DL, DT, dl):
    paths = [f'/home/datawork-lops-osi/aponte/margot/historical_coloc_ok/matchup/matchup_{l}.zarr'] + glob(os.path.join(f'/home/datawork-lops-osi/aponte/margot/historical_coloc_ok/cutoff_matchup/cutoff_matchup_{l}*.zarr'))
    ds = xr.merge([xr.open_dataset(path) for path in paths])[[comb[key] for key in comb]+['drogue_status']]
    ds['drifter_type'] = l.split('_')[0]
    ds['drogue_status'] = ds['drogue_status'].astype('bool')
    ds = ds.reset_coords(['alti___distance'])
    
    dfm = ds.to_dask_dataframe().set_index('obs')
    
    if drogue_status != 'both':
        dfm = dfm.where(dfm.drogue_status==drogue_status).dropna()
    if DL :
        dfm =dfm.where(dfm.alti___distance<DL).dropna()
    if DT : 
        dfm =dfm.where(dfm.alti___time_difference<DT).dropna()
        
    dfm = build_diagnostic(dfm, comb).drop(columns = ['drogue_status', 'alti___time_difference', 'drifter_type','time', 'lon', 'lat'])

    dfm["distbin"] = (dfm.alti___distance // dl) * dl
    dd = dfm.groupby(["distbin"]).mean()
    #count
    dnb = dfm.reset_index()[['obs', 'distbin']].groupby(["distbin"]).count().obs.compute().to_xarray()
    dsms = dd.compute().to_xarray().drop('alti___distance')

    #attrs
    for v in list(dsms.variables) :
        if v in ds :
            dsms[v].attrs = ds[v].attrs
    #merge
    dso = xr.merge([dsms, dnb.rename('nb_coloc_bin')])
    dso['drifter_sat_year']=l
    dso = dso.expand_dims('drifter_sat_year')
    dso = dso.set_coords('drifter_sat_year')
    # center lon, lat bins + reindex to have same for all
    dist_bins = np.arange(0, 200e3, dl)
    dso = dso.reindex({'distbin':dist_bins})
    dso['distbin'] = dso['distbin']+dl/2

    dso['id_comb'] = id_from_comb(comb)
    dso = dso.expand_dims('id_comb')
    dso = dso.set_coords('id_comb')
    
    ## ATTRS
    #for v in ds.keys() :
    #    ds[v].attrs =dsm[v].attrs
    return dso

def dist_dataset(combinations, drifter_type = 'both', drogue_status=True, DL=None, DT=None, dl=5):
    from histlib.cstes import labels
    DS =[]
    for l in labels :
        DS0 = []
        for comb in combinations :
            ds = onel_onecomb_dist_dataset(l, comb, drifter_type, drogue_status, DL, DT, dl)
            DS0.append(ds)
        DS.append(xr.concat(DS0, dim = 'id_comb'))
        print(l)
    return xr.concat(DS, dim='drifter_sat_year')
    


In [40]:
comb_alg = {'acc':'drifter_acc_x_25','cor':'drifter_coriolis_x_25','ggx':'alti_ggx_adt_filtered_ocean_tide_internal_tide','wd':'es_cstrio_z15_drifter_wd_x'}
comb_05 = {'acc':'drifter_acc_x_05','cor':'drifter_coriolis_x_05','ggx':'alti_ggx_adt_filtered_ocean_tide_internal_tide','wd':'es_cstrio_z15_drifter_wd_x'}
dsm = onel_onecomb_dist_dataset(labels[0], comb_alg, drifter_type = 'both', drogue_status=True, DL = None, DT=30*60, dl=5)
#ds = dist_dataset([comb_alg,comb_05], drifter_type = 'both', drogue_status=True, DL = None, DT=30*60, dl=5)

In [41]:
dsm

<xarray.Dataset>
Dimensions:           (distbin: 40000, drifter_sat_year: 1, id_comb: 1)
Coordinates:
  * distbin           (distbin) float64 2.5 7.5 12.5 17.5 ... 2e+05 2e+05 2e+05
  * drifter_sat_year  (drifter_sat_year) <U16 'gps_Jason-3_2020'
  * id_comb           (id_comb) <U115 'drifter_acc_x_25__drifter_coriolis_x_2...
Data variables: (12/33)
    obs               (id_comb, drifter_sat_year, distbin) float64 nan ... nan
    acc               (id_comb, drifter_sat_year, distbin) float64 nan ... nan
    cor               (id_comb, drifter_sat_year, distbin) float64 nan ... nan
    ggx               (id_comb, drifter_sat_year, distbin) float64 nan ... nan
    wd                (id_comb, drifter_sat_year, distbin) float64 nan ... nan
    s                 (id_comb, drifter_sat_year, distbin) float64 nan ... nan
    ...                ...
    COR               (id_comb, drifter_sat_year, distbin) float64 nan ... nan
    GGX               (id_comb, drifter_sat_year, distbin) float64 nan ... nan
    WD                (id_comb, drifter_sat_year, distbin) float64 nan ... nan
    S                 (id_comb, drifter_sat_year, distbin) float64 nan ... nan
    sigma             (id_comb, drifter_sat_year, distbin) float64 nan ... nan
    nb_coloc_bin      (id_comb, drifter_sat_year, distbin) float64 nan ... nan

________
# Time

In [42]:
def onel_onecomb_time_dataset(l, comb, drifter_type, drogue_status, DL, DT, dt):
    paths = [f'/home/datawork-lops-osi/aponte/margot/historical_coloc_ok/matchup/matchup_{l}.zarr'] + glob(os.path.join(f'/home/datawork-lops-osi/aponte/margot/historical_coloc_ok/cutoff_matchup/cutoff_matchup_{l}*.zarr'))
    ds = xr.merge([xr.open_dataset(path) for path in paths])[[comb[key] for key in comb]+['drogue_status']]
    ds['drifter_type'] = l.split('_')[0]
    ds['drogue_status'] = ds['drogue_status'].astype('bool')
    ds = ds.reset_coords(['alti___time_difference'])
    
    dfm = ds.to_dask_dataframe().set_index('obs')
    
    if drogue_status != 'both':
        dfm = dfm.where(dfm.drogue_status==drogue_status).dropna()
    if DL :
        dfm =dfm.where(dfm.alti___timeance<DL).dropna()
    if DT : 
        dfm =dfm.where(dfm.alti___time_difference<DT).dropna()
        
    dfm = build_diagnostic(dfm, comb).drop(columns = ['drogue_status', 'alti___timeance', 'drifter_type','time', 'lon', 'lat'])

    dfm["tbin"] = (dfm.alti___time_difference // dt) * dt
    dd = dfm.groupby(["timebin"]).mean()
    #count
    dnb = dfm.reset_index()[['obs', 'timebin']].groupby(["timebin"]).count().obs.compute().to_xarray()
    dsms = dd.compute().to_xarray().drop('alti___time_difference')

    #attrs
    for v in list(dsms.variables) :
        if v in ds :
            dsms[v].attrs = ds[v].attrs
    #merge
    dso = xr.merge([dsms, dnb.rename('nb_coloc_bin')])
    dso['drifter_sat_year']=l
    dso = dso.expand_dims('drifter_sat_year')
    dso = dso.set_coords('drifter_sat_year')
    # center lon, lat bins + reindex to have same for all
    time_bins = np.arange(0, 200e3, dl)
    dso = dso.reindex({'timebin':time_bins})
    dso['timebin'] = dso['timebin']+dl/2

    dso['id_comb'] = id_from_comb(comb)
    dso = dso.expand_dims('id_comb')
    dso = dso.set_coords('id_comb')
    
    ## ATTRS
    #for v in ds.keys() :
    #    ds[v].attrs =dsm[v].attrs
    return dso

def time_dataset(combinations, drifter_type = 'both', drogue_status=True, DL=None, DT=None, dl=5):
    from histlib.cstes import labels
    DS =[]
    for l in labels :
        DS0 = []
        for comb in combinations :
            ds = onel_onecomb_time_dataset(l, comb, drifter_type, drogue_status, DL, DT, dl)
            DS0.append(ds)
        DS.append(xr.concat(DS0, dim = 'id_comb'))
        print(l)
    return xr.concat(DS, dim='drifter_sat_year')
    


In [40]:
comb_alg = {'acc':'drifter_acc_x_25','cor':'drifter_coriolis_x_25','ggx':'alti_ggx_adt_filtered_ocean_tide_internal_tide','wd':'es_cstrio_z15_drifter_wd_x'}
comb_05 = {'acc':'drifter_acc_x_05','cor':'drifter_coriolis_x_05','ggx':'alti_ggx_adt_filtered_ocean_tide_internal_tide','wd':'es_cstrio_z15_drifter_wd_x'}
dsm = onel_onecomb_time_dataset(labels[0], comb_alg, drifter_type = 'both', drogue_status=True, DL = 25e3, DT=None, dt=5*60)
#ds = time_dataset([comb_alg,comb_05], drifter_type = 'both', drogue_status=True, DL = 25e3, DT=None, dl=5*60)